    
# ECON320-3 Assignment 5: James (Minkyu) Song 


In [2]:
# Script name: james_song_assignment_05.ipynb
# Purpose of script: Assignment 5 of ECON320 with Dr.O'Connell
# Author: James (Minkyu) Song
# Date Created: September 27th 2022
# --------------------------
# Notes: NA
# ---------------------------


# Assignment 5


## Part I

* Objective: Introduction to Multiple Regression Analysis and demonstrate your understanding of the mechanics.*

* Reference: Wooldridge (7th edition), Chapter 3, Section 1*

Using all of the previous up to this point, how close is your coefficient to the population value? Is it the same or different? Why or why not, do you think?


In [3]:
import math

import pandas as pd
import numpy as np

In [4]:
birthdaystring = "04022000" 
rng = np.random.RandomState(int(birthdaystring))

In [5]:
# function that finds the variance using my birthday,
# note that "scale" parameter in rng.normal is standard deviation
# so we have to square root the result to use it in rng.normal, since I return the Variance.
def birthday_str_variance_srm(birthday_string):
    for b in list(birthday_string):
        if b != "0":
            var_x= int(b)
            break
    for revb in reversed(list(birthday_string)):
        if revb != "0":
            var_u = int(revb)
            break
    return var_x, var_u

def generate_srm_birhtday(birthday_string, rng, x_mean, u_mean, size): # x_mean = 10, u_mean =0
    #beta0 is addition of 3rd and 4th digit of my birthday
    pop_beta0 = int(birthday_string[2]) +int(birthday_string[3])
    #beta1 is addition of 1st and 2nd digit of my birthday
    pop_beta1= int(birthday_string[0]) + int(birthday_string[1])
    #set scale, scale should be std, so sqrt the result
    var_x, var_u = birthday_str_variance_srm(birthday_string)
    scale_x = math.sqrt(var_x)
    scale_u = math.sqrt(var_u)

    x = rng.normal(loc = x_mean, scale= scale_x, size = size)
    print(f"x is normally distributed with a mean of {x_mean} and variance of {round(var_x,3)}")
    u = rng.normal(loc = u_mean, scale= scale_u, size = size)
    print(f"u is normally distributed with a mean of {u_mean} and variance of {round(var_u,3)}")

    y= pop_beta0+ pop_beta1*x+u
    print(f"y = {int(pop_beta0)} + {int(pop_beta1)}*x + u ")
    gdata = {"x": x, "u":u, "y":y}
    df = pd.DataFrame(data= gdata)
    return df, pop_beta0, pop_beta1

In [6]:
srm_df, pop_b0, pop_b1 = generate_srm_birhtday(birthdaystring, rng, 10, 0, 1000000)
print(f"population b_0 is: {pop_b0}")
print(f"population b_1 is: {pop_b1}")
srm_df.head()

x is normally distributed with a mean of 10 and variance of 4
u is normally distributed with a mean of 0 and variance of 2
y = 2 + 4*x + u 
population b_0 is: 2
population b_1 is: 4


,x,u,y
0,9.762285,0.670997,41.720135
1,11.905537,-2.301066,47.321083
2,10.940033,-1.951259,43.808873
3,12.577342,-1.658884,50.650484
4,12.484329,-0.117752,51.819565


In [7]:
# creating sample, referenced from previous assignmnet submissions
# and lab lectures

sample_srm_df = srm_df.sample(n = 1000, random_state= int(birthdaystring))

def sample_betas(sample_x, sample_y):
    covxy= np.cov(sample_x, sample_y)[1,0]
    print("covariance is: "+ str(covxy))
    varx= sample_x.var()
    beta1_sample = covxy/varx
    beta0_sample = sample_y.mean() - (beta1_sample* sample_x.mean())
    print(f'sample b_0: {beta0_sample}')
    print(f'sample b_1: {beta1_sample}')

    return beta0_sample, beta1_sample

def predict_y(sample_x, sample_y):
    beta0_sample, beta1_sample = sample_betas(sample_x, sample_y)
    y_pred = beta1_sample *sample_x +beta0_sample
    return y_pred, beta0_sample, beta1_sample

def residuals(sample_y, sample_y_pred):
    return sample_y - sample_y_pred

sample_srm_df['y_pred'], sample_b0, sample_b1 = predict_y(sample_srm_df['x'], sample_srm_df['y'])
sample_srm_df['residuals'] = residuals(sample_srm_df['y'], sample_srm_df['y_pred'])
sample_srm_df.head()

covariance is: 16.692187152734462
sample b_0: 1.828873516428402
sample b_1: 4.010702423430497


,x,u,y,y_pred,residuals
345468,13.536626,2.000103,58.146605,56.120250,2.026354
92519,10.131876,1.789076,44.316580,42.464813,1.851766
724467,10.597634,-0.279561,44.110974,44.332829,-0.221855
357633,11.083328,-0.599694,45.733616,46.280802,-0.547186
211894,11.061429,-3.419116,42.826599,46.192973,-3.366374


In [8]:
print("-"*30)
print("population coefficients")
print("-"*30)
print(f"population b_0: {pop_b0}")
print(f"population b_1: {pop_b1}")
print()
print("-"*30)
print("sample coefficients")
print("-"*30)
print(f"sample b_0: {round(sample_b0,3)}")
print(f"sample b_1: {round(sample_b1,3)}")
print()
print("-"*30)
print("population - sample coefficients")
print("-"*30)
print(f"pop b_0 - sample_b0: {round(pop_b0-sample_b0, 3)}")
print(f"pop b_1 - sample_b1: {round(pop_b1-sample_b1, 3)}")


------------------------------
population coefficients
------------------------------
population b_0: 2
population b_1: 4

------------------------------
sample coefficients
------------------------------
sample b_0: 1.829
sample b_1: 4.011

------------------------------
population - sample coefficients
------------------------------
pop b_0 - sample_b0: 0.171
pop b_1 - sample_b1: -0.011


In [11]:
# The values I got are close is your coefficient to the population value but different.
# This is because our sample coefficient is the closest estimate we can get from the underlying population coefficients.
# Also since this Data was generated through my birthday, our x variance was 4, it makes our sample a bit less precise
# Since we take a sample, we can't get the exact values, but we can get closer as we increase our sample size as well.

sample_srm_df_10k = srm_df.sample(n = 10000, random_state= int(birthdaystring))
sample_srm_df_10k['y_pred'], sample_b0_10k, sample_b1_10k = predict_y(sample_srm_df_10k['x'], sample_srm_df_10k['y'])
sample_srm_df_10k['residuals'] = residuals(sample_srm_df_10k['y'], sample_srm_df_10k['y_pred'])

print("-"*30)
print("sample coefficients_10k")
print("-"*30)
print(f"10ksample b_0: {round(sample_b0_10k,3)}")
print(f"10ksample b_1: {round(sample_b1_10k,3)}")
print()
print("-"*30)
print("population - sample coefficients (1k)")
print("-"*30)
print(f"pop b_0 - sample_b0: {round(pop_b0-sample_b0, 3)}")
print(f"pop b_1 - sample_b1: {round(pop_b1-sample_b1, 3)}")
print()
print("-"*30)
print("population - sample coefficients (10k)")
print("-"*30)
print(f"pop b_0 - sample_b0: {round(pop_b0-sample_b0_10k, 3)}")
print(f"pop b_1 - sample_b1: {round(pop_b1-sample_b1_10k, 3)}")

covariance is: 15.782376299515015
sample b_0: 1.953894058883563
sample b_1: 4.005627616852894
------------------------------
sample coefficients_10k
------------------------------
10ksample b_0: 1.954
10ksample b_1: 4.006

------------------------------
population - sample coefficients (1k)
------------------------------
pop b_0 - sample_b0: 0.171
pop b_1 - sample_b1: -0.011

------------------------------
population - sample coefficients (10k)
------------------------------
pop b_0 - sample_b0: 0.046
pop b_1 - sample_b1: -0.006



## Part II

*Objective: Motivation for Multiple Regression and build your own model.*

*Reference: Wooldridge (7th edition), Chapter 3, Section 1*

Perform a data generating process of $$y = \beta_0 + \beta_1 * x_1 +  \beta_2 * x_2 + u$$ where $\beta_0$ is the sum of the 3rd and 4th digits of your birthday, $\beta_1$ is the sum of the 1st and second digits of your birthday, $\beta_2$ is the sum of the 5th and 6th digits of your birthday,, $x$ is distributed normal with mean of 10 and with variance of the first non-zero digit of your birthday, and U is (independently) distributed normal with mean 0 and variance of the last non-zero digit of your birthday, and X2 is ~N(last nonzero of birthday, sum first two).

Perform this process for a "population" of 1 million observations.


In [12]:
# refactored for mrm
def birthday_str_variance_mrm(birthday_string):
    for b in list(birthday_string):
        if b != "0":
            var_x1= int(b)
            break
    for revb in reversed(list(birthday_string)):
        if revb != "0":
            var_u = int(revb)
            break
    var_x2 = int(birthday_string[0])+int(birthday_string[1])
    return var_x1, var_x2, var_u

# refactored for mrm
def generate_mrm_birhtday(birthday_string, rng, x1_mean, x2_mean, u_mean, size): # x_mean = 10, u_mean =0
    #beta0 is addition of 3rd and 4th digit of my birthday
    pop_beta0 = int(birthday_string[2]) +int(birthday_string[3])
    #beta1 is addition of 1st and 2nd digit of my birthday
    pop_beta1= int(birthday_string[0]) + int(birthday_string[1])
    #beta2 is addition of 5th and 6th digit of my birthday
    pop_beta2 = int(birthday_string[4])+int(birthday_string[5])
    #set scale, scale should be std, so sqrt the result
    var_x1, var_x2, var_u = birthday_str_variance_mrm(birthday_string)
    scale_x1 = math.sqrt(var_x1)
    scale_x2 = math.sqrt(var_x2)
    scale_u = math.sqrt(var_u)

    x_1 = rng.normal(loc = x1_mean, scale= scale_x1, size = size)
    print(f"x is normally distributed with a mean of {x1_mean} and variance of {round(var_x1,3)}")
    x_2 = rng.normal(loc = x2_mean, scale= scale_x2, size = size)
    print(f"x is normally distributed with a mean of {x2_mean} and variance of {round(var_x2,3)}")
    u = rng.normal(loc = u_mean, scale= scale_u, size = size)
    print(f"u is normally distributed with a mean of {u_mean} and variance of {round(var_u,3)}")

    y= pop_beta0 + pop_beta1*x_1 + pop_beta2*x_2 + u
    print(f"y = {int(pop_beta0)} + {int(pop_beta1)}*x_1 + {int(pop_beta2)}*x_2 + u ")
    gdata = {"x_1": x_1, "x_2": x_2, "u":u, "y":y}
    df = pd.DataFrame(data= gdata)
    return df, pop_beta0, pop_beta1, pop_beta2

In [13]:
mrm_df, pop_b0_mrm, pop_b1_mrm, pop_b2_mrm = generate_mrm_birhtday(birthdaystring, rng, 10, 2, 0, 1000000)
print(f"population b_0 is: {pop_b0_mrm}")
print(f"population b_1 is: {pop_b1_mrm}")
print(f"population b_2 is: {pop_b2_mrm}")

mrm_df[:10]

x is normally distributed with a mean of 10 and variance of 4
x is normally distributed with a mean of 2 and variance of 4
u is normally distributed with a mean of 0 and variance of 2
y = 2 + 4*x_1 + 2*x_2 + u 
population b_0 is: 2
population b_1 is: 4
population b_2 is: 2


,x_1,x_2,u,y
0,9.868464,0.520997,-0.818329,41.697519
1,10.842267,4.523183,0.859774,55.275206
2,12.346856,1.345217,2.724792,56.802649
3,6.005021,2.418919,1.309748,32.167670
4,10.321140,3.715284,2.746046,53.461174
5,8.591196,1.503948,1.408724,40.781402
6,11.200151,1.719009,0.583276,50.821900
7,8.563928,-0.499947,0.068980,35.324798
8,7.788651,-1.082705,-2.220774,28.768423
9,10.034535,-0.425496,0.621076,41.908225


In [14]:
mrm_df.describe().apply(lambda x: round(x, 3))

,x_1,x_2,u,y
count,1000000.000,1000000.000,1000000.000,1000000.000
mean,10.002,2.000,0.002,46.010
std,2.000,1.999,1.413,9.051
min,0.307,-7.318,-7.643,1.837
25%,8.653,0.650,-0.951,39.889
50%,10.000,2.001,0.002,46.022
75%,11.354,3.348,0.954,52.127
max,19.487,11.567,6.550,88.547
